 ### Problem Statement : Develop a sentiment analysis model to analyze customer reviews of products or services.

In [56]:
import pandas as pd
import numpy as np

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [60]:
df=pd.read_csv("Dataset-SA.csv")

In [62]:
df

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral
...,...,...,...,...,...,...
205047,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,must buy!,good product,positive
205048,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,super!,nice,positive
205049,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3,nice,very nice and fast delivery,positive
205050,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,just wow!,awesome product,positive


In [64]:
df = df[['Review', 'Sentiment']]
df.dropna(inplace=True)


/tmp/ipykernel_6737/3242463063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [66]:
df

,Review,Sentiment
0,super!,positive
1,awesome,positive
2,fair,positive
3,useless product,negative
4,fair,neutral
...,...,...
205047,must buy!,positive
205048,super!,positive
205049,nice,positive
205050,just wow!,positive


In [68]:
df['Sentiment'] = df['Sentiment'].map({
    'negative': 0,
    'neutral': 1,
    'positive': 2
})


/tmp/ipykernel_6737/3662436508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentiment'] = df['Sentiment'].map({


In [70]:
df

,Review,Sentiment
0,super!,2
1,awesome,2
2,fair,2
3,useless product,0
4,fair,1
...,...,...
205047,must buy!,2
205048,super!,2
205049,nice,2
205050,just wow!,2


In [72]:
X = df['Review']
y = df['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [74]:
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=5000,
    ngram_range=(1, 2)
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [76]:
model=LogisticRegression(
    multi_class="multinomial",
    solver="lbfgs",
    class_weight="balanced"
)


model.fit(X_train_tfidf, y_train)


/home/admin1/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', multi_class='multinomial')

In [78]:
y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



Accuracy: 0.8118798159543212


In [80]:
print(classification_report(
    y_test,
    y_pred,
    target_names=['Negative', 'Neutral', 'Positive']
))


              precision    recall  f1-score   support

    Negative       0.84      0.75      0.79      4880
     Neutral       0.13      0.44      0.20      1762
    Positive       0.96      0.84      0.90     29436

    accuracy                           0.81     36078
   macro avg       0.64      0.68      0.63     36078
weighted avg       0.91      0.81      0.85     36078



In [82]:
model.predict_proba(X_test_tfidf)[:5]



array([[0.03432966, 0.33245655, 0.63321378],
       [0.27623485, 0.5301239 , 0.19364126],
       [0.07662124, 0.39054389, 0.53283487],
       [0.03157295, 0.20847664, 0.75995041],
       [0.07662124, 0.39054389, 0.53283487]])